In [1]:
import os 
# import numpy as np
import pandas as pd
# from scipy.stats import linregress
# from matplotlib import pyplot as plt

from temp_module import tempmb
from temp_module.constants import *

Build TS table

In [2]:
# Time series converted table build
def tec(df1, pt_dir):
    """Takes time series table and array of amplitude tables to convert to backscatter tables"""
    t, tamb, tatt, xii, xvv = (df1["temp"], df1["ambient_T"], df1["attitude_T"], df1["xmit_i"], df1["xmit_v"])

    tmb = tempmb.TempMb(0, 0, t, tatt, tamb)
    x = tmb.c_amp_scale()
    y = tmb.attitude_temp()

    xmb = tempmb.Xmit(xii, xvv)
    xcurr = xmb.xi()
    xvolt = xmb.xv()

    df_bld_dict = {"Amp_scl_fctr": x, "electr_T": y, "xmit_current": xcurr, "xmit_voltage":xvolt}
    dat1 = pd.DataFrame(df_bld_dict)
    dat2 = df1.join(dat1)

    dat2.to_csv(pt_dir, index=False) 
    g = pd.read_csv(pt_dir)
    return(g)


MB build function section

In [16]:
import glob

In [22]:
# function version for mb build
def mb_build(amp_table, df_ts, pt_dir):
    """Takes time series table and array of amplitude tables to convert to backscatter tables"""
    tmp, tamb, tatt = (df_ts["temp"], df_ts["ambient_T"], df_ts["attitude_T"])

    # loop over amp df and separate each column into series
    df_mb = []
    df_mb.append(amp_table['time'])

    for x in amp_table:
        """Build mb table by operating mb function column by column"""
        # loop over string-type depths, ignoring time
        if x.startswith('time'):
            pass
        else:
            dep = float(x)
            # function to input only amplitude series by depth
            def tp_mb(y):
                tm = tempmb.TempMb(dep, y, tmp, tatt, tamb).measured_backscatter()
                return (tm)
            
            # set column as series for processing
            amp_x = amp_table[x]
            # run the function on the series
            mb_series_x = tp_mb(amp_x)
            # set new series as dataframe component with corresponding headings
            mb_df_x = pd.Series(mb_series_x, name=x)
            # add processed mb series to a list
            df_mb.append(mb_df_x)

    # Build and return mb table 
    df = pd.concat(df_mb, axis=1)

    # apply WR near zone corrections; TODO turn this off or verify explicitly
    df.iloc[:,1] += 1
    df.iloc[:,2] += 0.5

    # save tp csv
    df.to_csv(pt_dir, index=False) 
    # read csv back in for fun
    g = pd.read_csv(pt_dir)
    return (g)

Define data, directories, run functions to build tables

In [4]:
# set up directory to store data
test_dir = "/home/mpoe/adcp_habs/repos/test_dir/test_data"
try:
    os.mkdir(test_dir)
except OSError as error:
    print(error)

[Errno 17] File exists: '/home/mpoe/adcp_habs/repos/test_dir/test_data'


In [3]:
# directory = "/home/mpoe/adcp_habs/data/adcp_data_tables/OWS19002/OWS19002_"
ident = "SEN19" 
directory = "/home/mpoe/adcp_habs/data/adcp_tables_stacked/" + ident + "/" + ident + "_"

Build the time series table

In [4]:
temp_check = pd.read_csv(directory + "table_time_series.csv")
pnt_dir1 = directory + "converted_time_series.csv"


In [5]:
test_ts = tec(temp_check, pnt_dir1)
test_ts

,time,roll,pitch,heading,temp,xmit_i,xmit_v,attitude_T,ambient_T,card_adj,Amp_scl_fctr,electr_T,xmit_current,xmit_voltage
0,2019-05-15 15:30:00,-0.89,-3.080371,68.35,7.66,71,55,107,111,2019-05-15 15:30:00,0.454020,7.383964,0.813021,20.936685
1,2019-05-15 15:35:00,-0.40,-0.790019,69.18,7.67,68,53,106,111,2019-05-15 15:35:00,0.454117,7.324505,0.778668,20.175351
2,2019-05-15 15:40:00,-0.08,-0.820001,66.88,7.63,68,53,106,111,2019-05-15 15:40:00,0.454178,7.286306,0.778668,20.175351
3,2019-05-15 15:45:00,0.02,-0.830000,70.36,7.61,72,56,105,111,2019-05-15 15:45:00,0.454321,7.198649,0.824472,21.317352
4,2019-05-15 15:50:00,-0.09,-0.820001,73.03,7.60,73,56,105,111,2019-05-15 15:50:00,0.454336,7.189189,0.835923,21.317352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40188,2019-10-03 03:30:56,-0.35,-0.880016,71.69,18.81,1,3,85,92,2019-10-02 18:32:56,0.438393,17.378804,0.011451,1.142001
40189,2019-10-03 03:35:56,-0.34,-0.880015,70.68,18.80,1,2,85,92,2019-10-02 18:37:56,0.438407,17.369565,0.011451,0.761334
40190,2019-10-03 03:40:56,-0.37,-0.850018,69.89,18.79,1,2,85,92,2019-10-02 18:42:56,0.438421,17.360326,0.011451,0.761334
40191,2019-10-03 03:45:56,-0.41,-0.890023,73.47,18.78,1,1,86,92,2019-10-02 18:47:56,0.438127,17.555217,0.011451,0.380667


Build the mb table

In [23]:
# pull time series and amplitude tables 
cats = ['beam1', 'beam2', 'beam3', 'beam4', 'avg']

#generate mb tables 
for i in cats:
    amp_check = pd.read_csv(directory + "amp_" + i + ".csv") 
    pnt_dir_mb = directory + "mb_" + i + ".csv"
    test_df = mb_build(amp_check, temp_check, pnt_dir_mb)
    # print(len(test_df), '\n' , test_df.iloc[10000, :])


40193 
 time     2019-06-19 23:13:00
1.61               55.282321
2.61               56.133778
3.61               54.726852
4.61               59.405326
5.61               54.134637
6.61               52.559468
7.61               49.937121
8.61               50.239423
9.61               49.999613
10.61              48.800769
11.61              47.531269
12.61              47.949609
13.61              46.569994
14.61              46.020277
15.61              48.490156
16.61              47.868779
17.61              47.217681
18.61              47.413695
19.61              48.459527
20.61              48.173402
21.61              49.178216
22.61              49.728427
23.61              50.262359
24.61                49.9077
25.61              49.976073
26.61              51.342147
27.61              53.133213
28.61              66.270537
29.61              87.260321
30.61             104.745979
Name: 10000, dtype: object
40193 
 time     2019-06-19 23:13:00
1.61               59.650497
